![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [7]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "./Dados/features.txt"
filename_labels = "./Dados/activity_labels.txt"

filename_subtrain = "./Dados/train/subject_train.txt"
filename_xtrain = "./Dados/train/X_train.txt"
filename_ytrain = "./Dados/train/y_train.txt"

filename_subtest = "./Dados/test/subject_test.txt"
ffilename_xtest = "./Dados/test/X_test.txt"
filename_ytest = "./Dados/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'],sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\marce\AppData\Local\Temp\ipykernel_19732\2224838783.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
C:\Users\marce\AppData\Local\Temp\ipykernel_19732\2224838783.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
C:\Users\marce\AppData\Local\Temp\ipykernel_19732\2224838783.py:29: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:
%%time
tree = DecisionTreeClassifier(ccp_alpha=0.001)
tree.fit(X_train, y_train)

CPU times: total: 3.16 s
Wall time: 5.12 s


DecisionTreeClassifier(ccp_alpha=0.001)

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [13]:
import time

# Realizar PCA com uma única componente
prcomp = PCA(n_components=1).fit(X_train)
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Treinar a árvore de decisão com a componente principal
tree = DecisionTreeClassifier(ccp_alpha=0.001)

# Medir o tempo de treinamento
start_time_train = time.time()
tree.fit(pc_treino, y_train)
end_time_train = time.time()
train_time = end_time_train - start_time_train

# Fazer previsões
y_train_pred = tree.predict(pc_treino)
y_test_pred = tree.predict(pc_teste)

# Avaliar a acurácia nas bases de treinamento e teste
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Medir o tempo de predição
start_time_test = time.time()
y_test_pred = tree.predict(pc_teste)
end_time_test = time.time()
test_time = end_time_test - start_time_test

# Resultados
print(f'Acurácia na base de treinamento: {train_accuracy:.4f}')
print(f'Acurácia na base de teste: {test_accuracy:.4f}')
print(f'Tempo de processamento (treinamento): {train_time:.4f} segundos')
print(f'Tempo de processamento (teste): {test_time:.4f} segundos')

Acurácia na base de treinamento: 0.4997
Acurácia na base de teste: 0.4571
Tempo de processamento (treinamento): 0.0432 segundos
Tempo de processamento (teste): 0.0015 segundos


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [15]:
%%time

# Lista de quantidades de componentes principais para testar
n_components_list = [1, 2, 5, 10, 50]

# Armazenar os resultados
results = []

# Loop para testar diferentes quantidades de componentes principais
for n_components in n_components_list:
    print(f'Testando com {n_components} componentes principais...')
    
    # Realizar PCA com a quantidade específica de componentes
    prcomp = PCA(n_components=n_components).fit(X_train)
    pc_treino = prcomp.transform(X_train)
    pc_teste = prcomp.transform(X_test)
    
    # Treinar a árvore de decisão com as componentes principais
    tree = DecisionTreeClassifier(ccp_alpha=0.001)
    
    # Medir o tempo de treinamento
    start_time_train = time.time()
    tree.fit(pc_treino, y_train)
    end_time_train = time.time()
    train_time = end_time_train - start_time_train
    
    # Fazer previsões
    y_train_pred = tree.predict(pc_treino)
    y_test_pred = tree.predict(pc_teste)
    
    # Avaliar a acurácia nas bases de treinamento e teste
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Medir o tempo de predição
    start_time_test = time.time()
    y_test_pred = tree.predict(pc_teste)
    end_time_test = time.time()
    test_time = end_time_test - start_time_test
    
    # Armazenar os resultados
    results.append({
        'n_components': n_components,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'train_time': train_time,
        'test_time': test_time
    })

# Exibir os resultados
for result in results:
    print(f"Componentes: {result['n_components']}")
    print(f"Acurácia na base de treinamento: {result['train_accuracy']:.4f}")
    print(f"Acurácia na base de teste: {result['test_accuracy']:.4f}")
    print(f"Tempo de processamento (treinamento): {result['train_time']:.4f} segundos")
    print(f"Tempo de processamento (teste): {result['test_time']:.4f} segundos")
    print('---')


Testando com 1 componentes principais...
Testando com 2 componentes principais...
Testando com 5 componentes principais...
Testando com 10 componentes principais...
Testando com 50 componentes principais...
Componentes: 1
Acurácia na base de treinamento: 0.4997
Acurácia na base de teste: 0.4571
Tempo de processamento (treinamento): 0.0385 segundos
Tempo de processamento (teste): 0.0000 segundos
---
Componentes: 2
Acurácia na base de treinamento: 0.6128
Acurácia na base de teste: 0.5847
Tempo de processamento (treinamento): 0.0356 segundos
Tempo de processamento (teste): 0.0000 segundos
---
Componentes: 5
Acurácia na base de treinamento: 0.8460
Acurácia na base de teste: 0.7889
Tempo de processamento (treinamento): 0.0497 segundos
Tempo de processamento (teste): 0.0000 segundos
---
Componentes: 10
Acurácia na base de treinamento: 0.8920
Acurácia na base de teste: 0.8232
Tempo de processamento (treinamento): 0.1118 segundos
Tempo de processamento (teste): 0.0000 segundos
---
Componentes:

## Conclua

- O que aconteceu com a acurácia?
<p>A acurácia aumenta com o número de componentes principais até 10 componentes, após o qual se estabiliza.</p>

- O que aconteceu com o tempo de processamento?
<p>O tempo de processamento de treinamento aumenta com o número de componentes principais, mas permanece razoável.</p>